# Import libraries and Module

In [0]:
import re
import torch
import warnings
import numpy as np
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import DataLoader

In [0]:
warnings.filterwarnings('ignore')

# Importing Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
cd drive/My\ Drive/dataset

[Errno 2] No such file or directory: 'drive/My Drive/dataset'
/content/drive/My Drive/dataset


In [6]:
!ls

cifar-10-python  processed_tag	  test.pickle
output.csv	 shakespeare.txt  train.pickle


In [0]:
with open('shakespeare.txt', 'r') as f:
    text = f.read()

In [8]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [0]:
def preprocessing_text(raw_text):
  
  # remove numerals
  temp_text = re.sub(r'[0-9]+','', raw_text)
  
  # convert to lower_case
  temp_text = "".join(list(map(lambda x : x.lower(), temp_text)))
  
  # remove new lines
#   temp_text = re.sub("[\n]+"," ", temp_text)
  
  # remove multiple spaces
  return re.sub(' +', ' ', temp_text)[2:]

In [0]:
text = preprocessing_text(text)

In [11]:
print(text[:1000])


 from fairest creatures we desire increase,
 that thereby beauty's rose might never die,
 but as the riper should by time decease,
 his tender heir might bear his memory:
 but thou contracted to thine own bright eyes,
 feed'st thy light's flame with self-substantial fuel,
 making a famine where abundance lies,
 thy self thy foe, to thy sweet self too cruel:
 thou that art now the world's fresh ornament,
 and only herald to the gaudy spring,
 within thine own bud buriest thy content,
 and tender churl mak'st waste in niggarding:
 pity the world, or else this glutton be,
 to eat the world's due, by the grave and thee.


 
 when forty winters shall besiege thy brow,
 and dig deep trenches in thy beauty's field,
 thy youth's proud livery so gazed on now,
 will be a tattered weed of small worth held: 
 then being asked, where all thy beauty lies,
 where all the treasure of thy lusty days;
 to say within thine own deep sunken eyes,
 were an all-eating shame, and thriftless praise.
 how much

# Vectorization and one hot encoding of text and character

In [0]:
char_dict = set(text)

In [13]:
len(char_dict)

48

In [0]:
int2char = dict(enumerate(char_dict))

In [0]:
char2int = {ch:idx for idx,ch in int2char.items()}

In [0]:
seq_length = 50

In [0]:
input_seq = []
target_seq = []
for idx in range(0,len(text)-1):
  input_seq.append(text[idx:idx+seq_length])
  target_seq.append(text[idx+1:idx+seq_length+1])

In [18]:
input_seq[:10]

['\n from fairest creatures we desire increase,\n that',
 ' from fairest creatures we desire increase,\n that ',
 'from fairest creatures we desire increase,\n that t',
 'rom fairest creatures we desire increase,\n that th',
 'om fairest creatures we desire increase,\n that the',
 'm fairest creatures we desire increase,\n that ther',
 ' fairest creatures we desire increase,\n that there',
 'fairest creatures we desire increase,\n that thereb',
 'airest creatures we desire increase,\n that thereby',
 'irest creatures we desire increase,\n that thereby ']

In [19]:
target_seq[:10]

[' from fairest creatures we desire increase,\n that ',
 'from fairest creatures we desire increase,\n that t',
 'rom fairest creatures we desire increase,\n that th',
 'om fairest creatures we desire increase,\n that the',
 'm fairest creatures we desire increase,\n that ther',
 ' fairest creatures we desire increase,\n that there',
 'fairest creatures we desire increase,\n that thereb',
 'airest creatures we desire increase,\n that thereby',
 'irest creatures we desire increase,\n that thereby ',
 'rest creatures we desire increase,\n that thereby b']

In [0]:
for idx in range(len(text)-1):
  input_seq[idx] = [char2int[character] for character in input_seq[idx]]
  target_seq[idx] = [char2int[character] for character in target_seq[idx]]

In [0]:
input_seq = input_seq[:pow(10,6)]

In [0]:
target_seq = target_seq[:pow(10,6)]

In [0]:
total_length = len(input_seq)
dict_size = len(char_dict)